In [148]:
import pandas as pd

df = pd.read_csv(r"C:\Users\rcrds\Desktop\Ironhack\Projects\Project 3\IronHack-w3-Modual1-Project3\Data\Clean\FIFA23_cleaned.csv")
clubs = set(df["Club"])

clubs

{'1. FC Heidenheim 1846',
 '1. FC Kaiserslautern',
 '1. FC Köln',
 '1. FC Magdeburg',
 '1. FC Nürnberg',
 '1. FC Saarbrücken',
 '1. FC Union Berlin',
 '1. FSV Mainz 05',
 '12 de Octubre FC',
 '9 de Octubre',
 'AC Ajaccio',
 'AC Arles Avignon',
 'AC Horsens',
 'AC Milan',
 'AC Mineros de Guayana',
 'AC Monza',
 'AC Sparta Praha',
 'AD Alcorcón',
 'ADO Den Haag',
 'AEK Athens',
 'AFC Astra Giurgiu',
 'AFC Bournemouth',
 'AFC Chindia Târgoviște',
 'AFC Hermannstadt',
 'AFC Richmond',
 'AFC UTA Arad',
 'AFC Wimbledon',
 'AIK',
 'AJ Auxerre',
 'APOEL Nicosia FC',
 'AS Monaco',
 'AS Nancy Lorraine',
 'AS Saint-Étienne',
 'ATK Mohun Bagan FC',
 'AZ Alkmaar',
 'Aalborg BK',
 'Aalesunds FK',
 'Aarhus GF',
 'Aberdeen',
 'Abha Club',
 'Academia Puerto Cabello',
 'Accrington Stanley',
 'Adana Demirspor',
 'Adelaide United',
 'Ajax',
 'Al Adalah',
 'Al Ahli',
 'Al Ain',
 'Al Ain FC',
 'Al Batin',
 'Al Faisaly',
 'Al Fateh',
 'Al Fayha',
 'Al Hazem',
 'Al Hilal',
 'Al Ittihad',
 'Al Khaleej',
 'Al N

In [149]:

url = "https://www.ea.com/able/news/fifa-23-all-leagues-clubs-teams-list"
from bs4 import BeautifulSoup
import requests

response = requests.get(url)
c = response.content
soup = BeautifulSoup(c)
tables = soup.find_all("ea-text")[1:-1]

my_dict = {}

for  table in tables:
    h2_tag = table.find("h2")
    if not h2_tag:
        continue

    h2_text = h2_tag.get_text().strip()

    current_league = "Rest of World"

    for element in table.find_all(['h3', 'p']):
        if element.name == 'h3':
            current_league = element.get_text(strip=True)
        elif element.name == 'p':
            if h2_text == "LALIGA SMARTBANK":
                current_league ="LALIGA SMARTBANK"
                h2_text = "SPAIN"
            team_name = element.get_text(strip=True).replace("*","").replace("(Authentic kit and crest with generic players)","")
            key = f"{h2_text}-{team_name}"
            if key in my_dict.keys():
                continue
            else:
                my_dict[key] = current_league
 
my_dict
import pandas as pd
my_dict

{'MEN’S FOOTBALL-Aldosivi': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Argentinos Jrs.': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Arsenal': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Atlético Tucumán': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Banfield': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Barracas Central': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Boca Juniors': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Central Córdoba': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Colón': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Defensa': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Estudiantes': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Gimnasia': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Godoy Cruz': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Huracán': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Independiente': 'LIGA PROFESIONAL DE FÚTBOL',
 'MEN’S FOOTBALL-Lanús': 'LIGA PROFESIONAL DE FÚTBOL',
 "MEN’S FOOTBALL-Newell's": 'LIGA PRO

In [150]:

df = pd.DataFrame(my_dict.items(), columns=["Country - Team","League"])
df["Country - Team"] = df["Country - Team"].str.replace("MEN’S FOOTBALL", "ARGENTINA")
df.head(110)
df[["Country", "Team"]] = df.pop("Country - Team").str.split("-", n=1, expand=True)
df

,League,Country,Team
0,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Aldosivi
1,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Argentinos Jrs.
2,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Arsenal
3,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Atlético Tucumán
4,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Banfield
...,...,...,...
754,Rest of World,WOMEN'S NATIONAL,Scotland
755,Rest of World,WOMEN'S NATIONAL,Spain
756,Rest of World,WOMEN'S NATIONAL,Sweden
757,Rest of World,WOMEN'S NATIONAL,United States


In [151]:
from rapidfuzz import process, fuzz
import numpy as np
df_matched = pd.DataFrame(columns=["Team", "Team (DB)", "Match Score"])
df_teams = df["Team"].tolist()

replacements = {
                "Sporting Club":"SC",
                "Football Club":"FC",
                "Kansas City":"KC",
                "Club Atletico": "CA",
                "Club Atlético": "CA",
                "Fútbol Club": "FC",
                "Futbol Club": "FC",
                "Clube Sport": "CS",
                "Aveyron Footbal": "AF",
                "Desportivo": "D",
                "Club de Regatas": "",
}

clubs2 = {
    pd.Series([club]).replace(replacements, regex=True).iloc[0]
    for club in clubs
}
for team in clubs2:
    match, score, idx = process.extractOne(
        team,
        df_teams,
        scorer=fuzz.token_sort_ratio
    )
    if score >= 75:
        df_matched.loc[len(df_matched)] = [
            match,   
            team,    
            score
        ]

display(df.head(30))
display(df.isnull().sum())
df = df.replace(r'^\s*$', np.nan, regex=True)
df=df.dropna(thresh=3)
#df.to_excel(r"C:\Users\rcrds\Desktop\Ironhack\Projects\Project 3\IronHack-w3-Modual1-Project3\Notebook\teams__leagues_matched.xlsx", index=False)
#df_matched.to_excel(r"C:\Users\rcrds\Desktop\Ironhack\Projects\Project 3\IronHack-w3-Modual1-Project3\Notebook\df_matched.xlsx", index=False)

,League,Country,Team
0,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Aldosivi
1,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Argentinos Jrs.
2,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Arsenal
3,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Atlético Tucumán
4,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Banfield
5,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Barracas Central
6,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Boca Juniors
7,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Central Córdoba
8,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Colón
9,LIGA PROFESIONAL DE FÚTBOL,ARGENTINA,Defensa


League     0
Country    0
Team       0
dtype: int64

In [152]:
df_matched.head(20)

,Team,Team (DB),Match Score
0,Lyngby BK,Lyngby BK,100.000000
1,Nashville SC,Nashville SC,100.000000
2,Bristol City,Bristol City,100.000000
3,Wuhan 3 Towns,Wuhan Three Towns,80.000000
4,Bologna,Bologna,100.000000
5,SD Eibar,SD Eibar,100.000000
6,River Plate,River Plate,100.000000
7,Sunderland,Sunderland,100.000000
8,Harrogate Town,Harrogate Town,100.000000
9,ATK Mohun Bagan,ATK Mohun Bagan FC,90.909091


In [153]:
len(clubs)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 737 entries, 0 to 757
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   League   737 non-null    object
 1   Country  737 non-null    object
 2   Team     737 non-null    object
dtypes: object(3)
memory usage: 23.0+ KB


In [154]:
len(df_matched)
piv = df.pivot_table(index="League", values="Team", aggfunc="count")
piv

,Team
League,
1A PRO LEAGUE,18
3. LIGA,20
3F SUPERLIGA,12
ALLSVENSKAN,16
BARCLAYS WOMEN’S SUPER LEAGUE,1
BUNDESLIGA,18
BUNDESLIGA 2,18
CINCH PREMIERSHIP,12
CONMEBOL LIBERTADORES,26


In [155]:
df[df["Team"]=="Arsenal"].tail(50)
display(len(df))
display(len(df_matched))
display(len(clubs))


737

548

927

In [156]:

df_merged  = pd.merge(df_matched,df,how="left",on="Team")
df_clubs = pd.DataFrame(clubs, columns=["Team (DB)"])

df_merged2 = pd.merge(df_merged,df_clubs,how="right",on="Team (DB)")
df_merged2

,Team,Team (DB),Match Score,League,Country
0,Nashville SC,Nashville SC,100.000000,MLS,USA/CANADA
1,Lyngby BK,Lyngby BK,100.000000,3F SUPERLIGA,DENMARK
2,Bristol City,Bristol City,100.000000,EFL CHAMPIONSHIP,ENGLAND
3,Wuhan 3 Towns,Wuhan Three Towns,80.000000,CSL,CHINA
4,NaN,Crotone,NaN,NaN,NaN
...,...,...,...,...,...
925,NaN,Urbs Reggina 1914,NaN,NaN,NaN
926,NaN,Sepsi OSK Sf. Gheorghe,NaN,NaN,NaN
927,Cerro Porteño,Club Cerro Porteño,83.870968,CONMEBOL LIBERTADORES,CONMEBOL***
928,NaN,Viktoria Žižkov,NaN,NaN,NaN


In [157]:
df_merged2["Match Score"] = df_merged2["Match Score"].fillna("Unmatched")
df_merged2 = df_merged2.fillna("Rest of World")
df_merged2

,Team,Team (DB),Match Score,League,Country
0,Nashville SC,Nashville SC,100.0,MLS,USA/CANADA
1,Lyngby BK,Lyngby BK,100.0,3F SUPERLIGA,DENMARK
2,Bristol City,Bristol City,100.0,EFL CHAMPIONSHIP,ENGLAND
3,Wuhan 3 Towns,Wuhan Three Towns,80.0,CSL,CHINA
4,Rest of World,Crotone,Unmatched,Rest of World,Rest of World
...,...,...,...,...,...
925,Rest of World,Urbs Reggina 1914,Unmatched,Rest of World,Rest of World
926,Rest of World,Sepsi OSK Sf. Gheorghe,Unmatched,Rest of World,Rest of World
927,Cerro Porteño,Club Cerro Porteño,83.870968,CONMEBOL LIBERTADORES,CONMEBOL***
928,Rest of World,Viktoria Žižkov,Unmatched,Rest of World,Rest of World


In [162]:
unmatched = df_merged2.loc[df_merged2["Match Score"]=="Unmatched"]

In [160]:

df_merged2.to_excel(r"C:\Users\rcrds\Desktop\Ironhack\Projects\Project 3\IronHack-w3-Modual1-Project3\Notebook\merged2.xlsx", index=False)

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

# Make a new column for links
unmatched["FIFA_Link"] = None

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/143.0.0.0 Safari/537.36"
}

for idx, row in unmatched.iterrows():
    team_name = row["Team (DB)"]
    query = f"{team_name} fifa index fifa 23"
    url = f"https://www.google.com/search?q={query.replace(' ', '+')}"

    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")

    # Select the first organic result
    first_result = soup.select_one("div.g a")
    if first_result:
        unmatched.at[idx, "FIFA_Link"] = first_result.get("href")
    else:
        unmatched.at[idx, "FIFA_Link"] = None

    # Optional: slow down requests to avoid blocking
    time.sleep(1)


C:\Users\rcrds\AppData\Local\Temp\ipykernel_38588\2529920082.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched["FIFA_Link"] = None


In [169]:

unmatched

,Team,Team (DB),Match Score,League,Country,FIFA_Link
4,Rest of World,Crotone,Unmatched,Rest of World,Rest of World,None
9,Rest of World,Ceará Sporting Club,Unmatched,Rest of World,Rest of World,None
10,Rest of World,Suwon Samsung Bluewings,Unmatched,Rest of World,Rest of World,None
12,Rest of World,Dalian Professional Football Club,Unmatched,Rest of World,Rest of World,None
15,Rest of World,Orlando City Soccer Club,Unmatched,Rest of World,Rest of World,None
...,...,...,...,...,...,...
924,Rest of World,Preston North End,Unmatched,Rest of World,Rest of World,None
925,Rest of World,Urbs Reggina 1914,Unmatched,Rest of World,Rest of World,None
926,Rest of World,Sepsi OSK Sf. Gheorghe,Unmatched,Rest of World,Rest of World,None
928,Rest of World,Viktoria Žižkov,Unmatched,Rest of World,Rest of World,None
